# ndarray

参考: https://www.cnblogs.com/mxnote/articles/17459015.html

## 引入依赖

In [ ]:
:dep ndarray = {version = "0.15.6"}

use ndarray::prelude::*;

## 一维数组

In [8]:
let arr1 = array![1., 2., 3., 4., 5., 6.];
println!("arr1: {}", arr1);

let arr2 = array![1., 2.2, 3.3, 4., 5., 6.];
let arr3 = arr1 + arr2;
println!("arr3: {}", arr3);

arr1: [1, 2, 3, 4, 5, 6]
arr3: [2, 4.2, 6.3, 8, 10, 12]


## 二维数组

In [16]:
let arr4 = array![
    [1., 2., 3.], 
    [ 4., 5., 6.]
];
let arr5 = Array::from_elem((2, 1), 1.);
let arr6 = arr4 + arr5;
println!("arr6:\n{}", arr6);

let arr7 =  Array::<f64, _>::zeros(arr6.raw_dim());
let arr8 = arr6 * arr7;
println!("arr8: \n{}", arr8);

arr6:
[[2, 3, 4],
 [5, 6, 7]]
arr8: 
[[0, 0, 0],
 [0, 0, 0]]


In [29]:
let identity: Array2<f64> = Array::eye(3);
println!("i: \n{}", &identity);

let arr9 = array![
    [1., 2., 3.],
    [4., 5., 6.],
    [7., 8., 9.]
];
println!("arr9:\n{}", &arr9);
let arr10 = &arr9 * &identity;
println!("arr9 * i = \n{}", arr10);

let arr11 = arr10.dot(&identity);
println!("arr11: {}", &arr11);

i: 
[[1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]
arr9:
[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]
arr9 * i = 
[[1, 0, 0],
 [0, 5, 0],
 [0, 0, 9]]
arr11: [[1, 0, 0],
 [0, 5, 0],
 [0, 0, 9]]


## 随机矩阵

In [26]:
:dep ndarray-rand={version = "0.14.0"}

use ndarray_rand::{RandomExt, SamplingStrategy};
use ndarray_rand::rand_distr::Uniform;

In [30]:
let arr13 = Array::random((2, 5), Uniform::new(0., 10.));
println!("{:5.2}", arr13);


[[ 1.63,  1.75,  7.67,  0.69,  4.47],


In [32]:
let arr14 = array![1., 2., 3., 4., 5., 6.];
let arr15 = arr14.sample_axis(Axis(0), 2, SamplingStrategy::WithoutReplacement);
println!("Sampling from:\t{}\nTwo elements:\t{}", arr14, arr15);


Sampling from:	[1, 2, 3, 4, 5, 6]


Two elements:	[2, 5]


In [33]:
use ndarray_rand::rand as rand;
use rand::seq::IteratorRandom;

In [35]:
let mut rng = rand::thread_rng();
let faces = "😀😎😐😕😠😢";
let arr16 = Array::from_shape_vec((2, 2), faces.chars().choose_multiple(&mut rng, 4)).unwrap();
println!("Sampling from:\t{}", faces);
println!("Elements:\n{}", arr16);


Sampling from:	😀😎😐😕😠😢
Elements:
[[😢, 😎],
 [😐, 😕]]


## 用ndarray-stats实现统计

In [37]:
use ndarray_rand::rand_distr::{Uniform, StandardNormal}; 

In [38]:
:dep ndarray-stats = {version="0.5.1"}
:dep noisy_float = {version="0.2.0"}

use ndarray_stats::HistogramExt;
use ndarray_stats::histogram::{strategies::Sqrt, GridBuilder};
use noisy_float::types::{N64, n64};

In [44]:
let arr17 = Array::<f64, _>::random_using((10000,2), StandardNormal, &mut rand::thread_rng());
let data = arr17.mapv(|e| n64(e));
let grid = GridBuilder::<Sqrt<N64>>::from_array(&data).unwrap().build();
let histogram = data.histogram(grid);
{
let histogram_matrix = histogram.counts();
let data = histogram_matrix.sum_axis(Axis(0));
println!("{}", data);

}

[3, 1, 0, 2, 1, 1, 3, 2, 3, 5, 5, 2, 5, 10, 12, 14, 15, 18, 22, 31, 46, 44, 42, 52, 54, 79, 79, 97, 110, 124, 119, 143, 154, 213, 201, 203, 220, 238, 232, 255, 290, 278, 271, 279, 300, 329, 307, 284, 285, 290, 308, 280, 289, 272, 257, 251, 244, 260, 224, 204, 197, 184, 169, 139, 121, 118, 115, 90, 65, 56, 62, 43, 47, 40, 31, 30, 21, 17, 22, 15, 11, 9, 9, 8, 1, 6, 1, 4, 1, 0, 2, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0]


()

In [41]:
:dep plotters = {version="0.3.4", default_features = false, features = ["evcxr", "all_series", "all_elements"]}

extern crate plotters;
use plotters::prelude::*;

In [45]:
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let histogram_matrix = histogram.counts();
    let data = histogram_matrix.sum_axis(Axis(0));
    let mut chart = ChartBuilder::on(&root)
        .set_label_area_size(LabelAreaPosition::Left, 40)
        .set_label_area_size(LabelAreaPosition::Bottom, 40)
        .caption("随机数分布图", ("sans-serif", 30))
        .build_cartesian_2d((0..data.len()).into_segmented(), 0..300)
        .unwrap();

    chart
        .configure_mesh()
        .disable_x_mesh()
        .bold_line_style(&WHITE.mix(0.3))
        .axis_desc_style(("sans-serif", 15))
        .draw()?;

    chart.draw_series((0..).zip(data.iter()).map(|(x, y)| {
        let x0 = SegmentValue::Exact(x);
        let x1 = SegmentValue::Exact(x + 1);
        let mut bar = Rectangle::new([(x0, 0), (x1, (*y) as i32)], RED.filled());
        bar.set_margin(0, 0, 1, 1);
        bar
    }))?;

    Ok(())
});
figure


随机数分布图
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
 
0
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40
 
 
 
50
 
 
 
60
 
 
 
70
 
 
 
80
 
 
 
90
 
 
 
100